In [5]:
import cv2 as cv
import numpy as np

# Question Extraction

In [6]:
def searchImg(haystack_img, needle_img, threshold):
    result = cv.matchTemplate(haystack_img, needle_img, cv.TM_CCOEFF_NORMED)

    threshold = 0.7
    locations = np.array(np.where(result >= threshold))

    # Filtering for when it gets the same square multiple times
    height = needle_img.shape[0]
    diffs = np.diff(locations[0])
    indices = np.where(diffs > height)[0]
    selected_indices = np.concatenate(([0], indices+1))
    locations = locations[:, selected_indices]

    locations = list(zip(*locations[::-1]))

    return locations

def trimWhitespace(image, output_path="trimmed_output.jpg"):

    # Convert to grayscale
    gray = cv.cvtColor(image, cv.COLOR_BGR2GRAY)

    # Apply a binary threshold to detect non-white regions
    _, thresh = cv.threshold(gray, 240, 255, cv.THRESH_BINARY_INV)

    # Find contours of the non-white regions
    contours, _ = cv.findContours(thresh, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

    if contours:
        # Get bounding box of the largest contour
        x, y, w, h = cv.boundingRect(np.concatenate(contours))

        # Crop the image using the bounding box
        cropped = image[y:y+h, x:x+w]

        # Save or return the cropped image
        cv.imwrite(output_path, cropped)
        return cropped

def extractRegions(regions, haystack_img, needle_width, needle_height):
    haystack_width = haystack_img.shape[1]
    haystack_height = haystack_img.shape[0]
    
    extracted_regions = []

    c = 0
    for reg_start, reg_end in regions:
        top_left = (reg_start[0] + int(1.1*needle_width), reg_start[1] - 4*needle_height//5)
        bottom_right = (reg_start[0] + 3*haystack_width//4, reg_end[1] - needle_height//2)

        # Extract the region of interest (ROI) from the original image
        region_ = trimWhitespace(haystack_img[top_left[1]:bottom_right[1], top_left[0]:bottom_right[0]])
        
        extracted_regions.append(region_)
        
    return extracted_regions

def getWhiteIntervals(img):
    # Convert to grayscale
    img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

    # Apply a binary threshold to detect non-white regions
    _, img_thr = cv.threshold(img, 240, 255, cv.THRESH_BINARY_INV)

    intervals = []

    start = None
    end = None

    for c, line in enumerate(img_thr):

        if start is None:
            if np.sum(line) == 0:
                    start = c
        elif end is None:
            if np.sum(line) != 0:
                end = c-1
                intervals.append([start, end])
                start = None
                end = None

    return intervals

def splitLines(img):
    needle_height = img.shape[0]
    needle_width = img.shape[1]
    
    intervals = getWhiteIntervals(img)
    intervals.insert(0, [-1, 0])
    intervals.append([needle_height, -1])

    quest_intervals = [[intervals[inter][1], intervals[inter+1][0]] for inter in range(len(intervals)-1)]

    splitted_img = []

    for inter in quest_intervals:
        img_ = img[inter[0]:inter[1], 0:needle_width]
        img_ = trimWhitespace(img_)
        splitted_img.append(img_)

    return splitted_img

In [136]:
haystack_img = cv.imread("data/splitted_pages-jpg/output-005.jpg", cv.IMREAD_COLOR)
haystack_width = haystack_img.shape[1]
haystack_height = haystack_img.shape[0]

needle_img = cv.imread("data/Black_square.jpg", cv.IMREAD_COLOR)
needle_height = needle_img.shape[0]
needle_width = needle_img.shape[1]

questao_img = cv.imread("data/Questao.jpg", cv.IMREAD_UNCHANGED)
questao_img = cv.cvtColor(questao_img, cv.COLOR_BGRA2BGR)

locations_black_square = np.array(searchImg(haystack_img, needle_img, 0.7))
locations_questao = np.array(searchImg(haystack_img, questao_img, 0.7))

locations = locations_black_square + locations_questao

In [137]:
regions = []

for loc_square in locations_black_square:
    for loc_quest in locations_questao:
        if loc_quest[1] > loc_square[1]:
            regions.append([loc_square, loc_quest])
            break

# Adding from the last question to the end of the page
regions.append([locations_black_square[-1], np.array([haystack_width, haystack_height])])

regions

[[array([388, 693]), array([ 355, 1084])],
 [array([ 388, 1436]), array([ 355, 1827])],
 [array([ 388, 2600]), array([2480, 3508])]]

In [138]:
regions_extracted = extractRegions(regions, haystack_img, needle_width, needle_height)

In [139]:
cv.imwrite("test.jpg", haystack_img[regions[0][0][1]:regions[0][1][1], regions[0][0][0]:haystack_width])

True

In [140]:
quest_items = []
for region in regions_extracted:
    quest_items.append(splitLines(region))

In [141]:
new_haystack = haystack_img.copy()

In [148]:
for region, quest in zip(regions, quest_items):
    new_haystack[region[0][1]:region[1][1], region[0][0]:haystack_width] = (255, 255, 255)
    np.random.shuffle(quest)
    for c, quest_ in enumerate(quest):
        w = quest_.shape[1]
        h = quest_.shape[0]
        pad = 10

        new_haystack[region[0][1]+(c*h):region[0][1]+(h+c*h), region[0][0]:region[0][0]+w] = quest_
cv.imwrite("test.jpg", new_haystack)

True

In [ ]:
mask = np.ones_like(quest_) * 255  # White mask for transparency
cv.copyTo(quest_, mask, haystack_img[region[0][1]:region[0][1]+h, region[0][0]:region[0][0]+w])
